In [1]:
# Import Dependencies and modules
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
from spacy.lang.en import English
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from string import punctuation
from collections import Counter
from io import StringIO
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

import nltk
import glob
import errno
import os
import json

from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import models
from gensim.models import CoherenceModel, TfidfModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt


%matplotlib inline

# Load Data

In [2]:
json_data = pd.DataFrame(columns = ['author', 'text', 'title'])

In [3]:
# Define path to text files that contain the novels
path = '/Users/ayankarim/Documents/Thinkful/Bootcamp/Final Capstone Opinion Mining/opinion_mining/Notebook/Topic Modelling Working Set/files/*.json'
files = glob.glob(path)

all_jsons = []

for filenames in files:
    with open(filenames, 'r') as f:
        file = json.load(f)
        all_jsons.append(file)


In [4]:
for dicts in all_jsons:
    df = pd.DataFrame(dicts)
    json_data = json_data.append(df)

In [5]:
json_data.head()

,author,text,title
0,[\n\t\t\t\t\t\tJulian Chokkattu\t\t\t\t\t],"[<p>Google’s <a href=""https://store.google.com...",\n\t\tGoogle will announce hardware on October...
1,[\n\t\t\t\t\t\tChristian de Looper\t\t\t\t\t],"[<p>Google finally unveiled the new <a href=""h...",\n\t\tHere’s how to buy the new Google Pixel 3...
2,[\n\t\t\t\t\t\tSimon Hill\t\t\t\t\t],[<p>If you plan to buy one of Google’s <a href...,\n\t\tThe best Pixel 3 cases and covers\t
3,[\n\t\t\t\t\t\tSimon Hill\t\t\t\t\t],"[<p>As the developer of Android, Google turns ...",\n\t\tGoogle Pixel 3 vs. Pixel 2 vs. Pixel: Pi...
4,[\n\t\t\t\t\t\tSimon Hill\t\t\t\t\t],[<p>There are plenty of contenders in the <a h...,\n\t\tGoogle Pixel 3 vs. Samsung Galaxy S9: Wh...


# Clean Data

In [6]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!

    text = str(text).replace("\n", "")
    text = str(text).replace("\t", "")
    text = str(text).replace("\\n", "")
    text = str(text).replace("\\t", "")
    text = str(text).replace("\\", "")
    text = str(text).replace("xa0", " ")
    text = str(text).replace("\'", "")
    text = re.sub("<p>", "", str(text))
    text = re.sub("</p>", "", str(text))
    text = re.sub("</a>", "", str(text))
    text = re.sub('<[^>]+>', "", str(text)) 
    text = str(text).replace("\\u2019", "")
    text = str(text).replace("\\u2013", "")
    text = str(text).replace("\\u2018", "")
    text = str(text).replace("\\u00a0", "")
    text = str(text).replace("\\u00a3", "")
    text = str(text).replace("\u2014", "")
    text = str(text).replace("\u201d", "")
    text = str(text).replace("\u201c", "")
    return text


In [7]:
# Define function to clean text
def clean_text(df):
    # Convert lists to strings and remove brackets
    df['text'] = df['text'].astype(str)
    df['author'] = df['author'].astype(str)

    df['text'] = df['text'].map(lambda x: x.strip('[]'))
    df['author'] = df['author'].map(lambda x: x.strip('[]'))

    # Clean text
    df['text'] = df['text'].apply(lambda x: text_cleaner(x))
    df['title'] = df['title'].apply(lambda x: text_cleaner(x))
    df['author'] = df['author'].apply(lambda x: text_cleaner(x))

In [8]:
# Clean Text
clean_text(json_data)

In [9]:
# Reset index
json_data = json_data.reset_index()
json_data = json_data.drop(['index'], axis=1)

In [10]:
# Visualize dataframe
json_data.head()

,author,text,title
0,Julian Chokkattu,Google’s annual hardware launch event will tak...,"Google will announce hardware on October 9, ne..."
1,Christian de Looper,Google finally unveiled the new Google Pixel 3...,Here’s how to buy the new Google Pixel 3 and G...
2,Simon Hill,If you plan to buy one of Google’s Pixel 3 sma...,The best Pixel 3 cases and covers
3,Simon Hill,"As the developer of Android, Google turns out ...",Google Pixel 3 vs. Pixel 2 vs. Pixel: Picking ...
4,Simon Hill,There are plenty of contenders in the Android ...,Google Pixel 3 vs. Samsung Galaxy S9: Which sm...


# Pre-Process Data for NLP

In [11]:
# # Tokenize text
# df['text'] = df.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
# df['title'] = df.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)

# # Remove Stopwords, or keep it, might be important for aspect based semantics
# stop = stopwords.words('english')
# df['text'] = df['text'].apply(lambda x: [item for item in x if item not in stop])
# df['title'] = df['title'].apply(lambda x: [item for item in x if item not in stop])

# # Lowercase everything
# df['text'] = df['text'].astype(str)
# df['text'] = df['text'].apply(lambda x: x.lower())

# df['title'] = df['title'].astype(str)
# df['title'] = df['title'].apply(lambda x: x.lower())

# # remove all punctuations
# df['text'] = df['text'].apply(lambda x: ''.join(c for c in x if c not in punctuation))
# df['title'] = df['title'].apply(lambda x: ''.join(c for c in x if c not in punctuation))

In [12]:

spacy.load('en')
parser = English()

# Function to tokenize text
def tokenize(text):
    lda_tokens = []
    tokens  = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc = True))
        


In [13]:
data = list(json_data['text'])

data_words = list(sent_to_words(data))

In [14]:
print(data_words[:1])

[['google', 'annual', 'hardware', 'launch', 'event', 'will', 'take', 'place', 'on', 'october', 'in', 'new', 'york', 'city', 'the', 'company', 'sent', 'out', 'invites', 'to', 'media', 'including', 'digital', 'trends', 'confirming', 'the', 'date', 'which', 'had', 'leaked', 'in', 'august', 'the', 'date', 'and', 'venue', 'are', 'change', 'of', 'pace', 'considering', 'the', 'past', 'two', 'google', 'october', 'events', 'have', 'taken', 'place', 'in', 'san', 'francisco', 'on', 'october', 'the', 'company', 'is', 'widely', 'expected', 'to', 'launch', 'slew', 'of', 'hardware', 'products', 'ranging', 'from', 'smartphones', 'to', 'smart', 'home', 'devices', 'the', 'highlights', 'will', 'be', 'the', 'pixel', 'and', 'pixel', 'xl', 'successors', 'to', 'last', 'year', 'critically', 'acclaimed', 'pixel', 'and', 'pixel', 'xl', 'smartphones', 'there', 'have', 'been', 'an', 'alarmingly', 'high', 'number', 'of', 'leaks', 'for', 'the', 'pixel', 'series', 'and', 'if', 'true', 'we', 'know', 'quite', 'lot', '

In [15]:
# Create Bigrams and Trigrams

# Build the models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# fast way to get a sentece clubbed as a bigram/trigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [16]:
print(trigram_mod[bigram_mod[data_words[0]]])

['google', 'annual', 'hardware', 'launch', 'event', 'will', 'take', 'place', 'on', 'october', 'in', 'new', 'york', 'city', 'the', 'company', 'sent', 'out', 'invites', 'to', 'media', 'including', 'digital_trends', 'confirming', 'the', 'date', 'which', 'had', 'leaked', 'in', 'august', 'the', 'date', 'and', 'venue', 'are', 'change', 'of', 'pace', 'considering', 'the', 'past', 'two', 'google', 'october', 'events', 'have', 'taken', 'place', 'in', 'san', 'francisco', 'on', 'october', 'the', 'company', 'is', 'widely', 'expected', 'to', 'launch', 'slew', 'of', 'hardware', 'products', 'ranging', 'from', 'smartphones', 'to', 'smart', 'home', 'devices', 'the', 'highlights', 'will', 'be', 'the', 'pixel', 'and', 'pixel', 'xl', 'successors', 'to', 'last_year', 'critically', 'acclaimed', 'pixel', 'and', 'pixel', 'xl', 'smartphones', 'there', 'have', 'been', 'an', 'alarmingly', 'high', 'number', 'of', 'leaks', 'for', 'the', 'pixel', 'series', 'and', 'if', 'true', 'we', 'know', 'quite', 'lot', 'about',

In [17]:
# Remove Stopwords, make bigrams and lemmatize
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['google', 'annual', 'hardware', 'launch', 'event', 'take', 'place', 'october', 'new', 'york', 'city', 'company', 'send', 'invite', 'medium', 'include', 'digital_trend', 'confirm', 'date', 'leak', 'august', 'date', 'venue', 'change', 'pace', 'consider', 'october', 'event', 'take', 'place', 'san', 'francisco', 'october', 'company', 'widely', 'expect', 'launch', 'slay', 'hardware', 'product', 'range', 'smartphone', 'smart', 'home', 'device', 'highlight', 'pixel', 'pixel', 'successor', 'last_year', 'critically', 'acclaim', 'pixel', 'pixel', 'smartphone', 'alarmingly', 'high', 'number', 'leak', 'pixel', 'series', 'true', 'know', 'quite', 'lot', 'phone', 'may', 'due', 'carelessness', 'recently', 'someone', 'leave', 'pixel', 'lyft', 'separately', 'group', 'russia', 'claim', 'get', 'hand', 'shipment', 'pixel', 'smartphone', 'even', 'post', 'unbox', 'video', 'show', 'everything', 'get', 'box', 'pixel', 'expect', 'notch', 'design', 'cutout', 'top', 'screen', 'house', 'front_fac', 'camera', 'ga

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [36]:
gensim.corpora.MmCorpus.serialize('MmCorpusTest.mm', corpus)

In [56]:
# pixel, iphone, samsung, apple, essential, xs_max, huawei, galaxy, note, moto, oneplus, android, mate_pro, lg, sony, razor, htc

stop_words_lst = ['pixel', 'iphone', 'samsung', 'apple', 'essential', 'xs_max', 
                  'huawei', 'galaxy', 'note', 'moto', 'oneplus', 'android', 'mate_pro', 'lg', 'sony', 'razor', 'razor']


In [69]:
remove = '|'.join(stop_words_lst)

regex = re.compile(r'\b('+remove+r')\b', texts)

TypeError: unhashable type: 'list'

In [ ]:
import re
for w in stop_words_lst:
    pattern = r'\b'+w+r'\b'
    for doc in texts:
        texts = re.sub(pattern, '', doc)

In [45]:
type(data_lemmatized[:1][0][0])

str

In [20]:
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 3), (5, 1), (6, 3), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 2), (35, 2), (36, 2), (37, 3), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 3), (46, 2), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 2), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 3), (69, 3), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 3), (83, 2), (84, 1), (85, 4), (86, 1), (87, 1), (88, 3), (89, 14), (90, 1), (91, 2), (92, 1), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 3), (101, 1), (102, 1), (103, 1), (104, 1), (105, 3), (106, 1), (107, 1), (108, 1), (109, 1), (110, 2

In [21]:
# Build the lda topic model

lda = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                      id2word=id2word,
                                      num_topics=3, 
                                      random_state=100,
                                      update_every=1,
                                      chunksize=50,
                                      passes=100,
                                      alpha='auto',
                                      per_word_topics=True)

In [22]:
# Print the key words in the 10 topics
pprint(lda.print_topics())
doc_lda = lda[corpus]

[(0,
  '0.034*"phone" + 0.014*"camera" + 0.009*"also" + 0.008*"good" + '
  '0.008*"feature" + 0.008*"display" + 0.008*"new" + 0.007*"get" + '
  '0.007*"come" + 0.007*"samsung"'),
 (1,
  '0.036*"iphone" + 0.016*"apple" + 0.010*"phone" + 0.010*"case" + '
  '0.009*"essential" + 0.007*"new" + 0.006*"company" + 0.006*"device" + '
  '0.006*"make" + 0.005*"use"'),
 (2,
  '0.021*"pixel" + 0.014*"phone" + 0.012*"screen" + 0.012*"app" + 0.010*"use" '
  '+ 0.008*"camera" + 0.008*"photo" + 0.007*"feature" + 0.007*"get" + '
  '0.007*"new"')]


In [23]:
# Evaluate how good the model is

# Compute perplexity
print('\nPerplexity: ', lda.log_perplexity(corpus)) # Lower the better

# Compute Coherence
coherenece_model_lda = CoherenceModel(model=lda, texts=data_lemmatized, dictionary=id2word, coherence='c_v' )
coherence_lda = coherenece_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.299478807097001

Coherence Score:  0.2804162950425502


In [24]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word)
vis

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.038989 -0.112785       1        1  56.603699
2     -0.096381  0.084854       2        1  26.281483
1      0.135370  0.027931       3        1  17.114815, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
1196  Default  1088.000000        iphone  1088.000000  30.0000  30.0000
89    Default  1048.000000         pixel  1048.000000  29.0000  29.0000
1176  Default   527.000000         apple   527.000000  28.0000  28.0000
9     Default   497.000000           app   497.000000  27.0000  27.0000
4874  Default   447.000000     essential   447.000000  26.0000  26.0000
148   Default   282.000000          case   282.000000  25.0000  25.0000
586   Default   350.000000         photo   350.000000  24.0000  24.0000
665   Default   216.000000           tap   216.000000  23.0000  23.0000
17    Default  1240.000000        camera  1240.000000  22.0000  22.0000
105   Default   796.000000        screen   796.000000  21.0000  21.0000
466   Default   703.000000           use   703.000000  20.0000  20.0000
26    Default   284.000000       company   284.000000  19.0000  19.0000
771   Default   243.000000          mode   243.000000  18.0000  18.0000
1580  Default   319.000000          moto   319.000000  17.0000  17.0000
161   Default   769.000000       feature   769.000000  16.0000  16.0000
572   Default   311.000000     megapixel   311.000000  15.0000  15.0000
1571  Default   536.000000        huawei   536.000000  14.0000  14.0000
2334  Default   292.000000      mate_pro   292.000000  13.0000  13.0000
988   Default   130.000000       setting   130.000000  12.0000  12.0000
682   Default   264.000000        winner   264.000000  11.0000  11.0000
596   Default   252.000000           ram   252.000000  10.0000  10.0000
184   Default   280.000000         price   280.000000   9.0000   9.0000
956   Default    98.000000  notification    98.000000   8.0000   8.0000
241   Default   222.000000        button   222.000000   7.0000   7.0000
130   Default   197.000000          turn   197.000000   6.0000   6.0000
340   Default   358.000000          lens   358.000000   5.0000   5.0000
61    Default   189.000000          home   189.000000   4.0000   4.0000
268   Default   219.000000          dual   219.000000   3.0000   3.0000
196   Default   193.000000           set   193.000000   2.0000   2.0000
578   Default   360.000000          note   360.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
98     Topic3    50.617805        report    92.242592   1.1651  -6.0171
2621   Topic3    59.136745        xs_max   114.883896   1.1012  -5.8615
267    Topic3    51.445980          drop   106.869766   1.0341  -6.0009
854    Topic3    46.363640        accord    96.355377   1.0337  -6.1049
805    Topic3    39.720829          sale    75.768204   1.1194  -6.2595
35     Topic3   126.814392        device   557.563782   0.2844  -5.0987
808    Topic3    46.606819          sell   107.363609   0.9308  -6.0997
78     Topic3   119.680824          make   626.049622   0.1106  -5.1566
82     Topic3   139.321152           new   871.597534  -0.0683  -5.0046
256    Topic3    61.337017         could   214.904617   0.5114  -5.8250
466    Topic3   111.549309           use   703.559570  -0.0765  -5.2269
447    Topic3    47.666637          tech   133.708054   0.7338  -6.0772
1233   Topic3    69.989677           say   299.534302   0.3113  -5.6931
88     Topic3   204.905319         phone  2972.949219  -0.9095  -4.6189
79     Topic3    83.942177           may   461.326813   0.0612  -5.5113
116    Topic3    78.007973    smartphone   396.581451   0.1392  -5.5846
4      Topic3   109.293938          also   886.702148  -0.3282  -5.2474
478    Topic3    67.073006          work   289.180817   0.3040  -5.7356
105    Topic3  

In [33]:
# Mallet LDA

mallet_path = '/Users/ayankarim/mallet-2.0.8/bin/mallet'

ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=3, id2word=id2word, iterations=1000, random_seed=100 )

In [34]:
# Show topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(0,
  [('phone', 0.04289988378705833),
   ('camera', 0.03160002967138936),
   ('pixel', 0.030240090992260712),
   ('huawei', 0.014365897683159014),
   ('feature', 0.013821922211507553),
   ('good', 0.011596568009297035),
   ('display', 0.01092896174863388),
   ('moto', 0.010879509433029202),
   ('lens', 0.009470118438295873),
   ('mate_pro', 0.0087283337042257)]),
 (1,
  [('iphone', 0.03962426407355957),
   ('samsung', 0.024012700932724746),
   ('phone', 0.019448303234768802),
   ('galaxy', 0.018952173050208375),
   ('apple', 0.01885294701329629),
   ('device', 0.013031686181120593),
   ('make', 0.010650261295230536),
   ('company', 0.00988952834557121),
   ('essential', 0.009724151617384401),
   ('note', 0.009029569358999802)]),
 (2,
  [('screen', 0.028379924799738433),
   ('phone', 0.021383030897498774),
   ('app', 0.017361451692005887),
   ('case', 0.010593428151054438),
   ('time', 0.00938368481281674),
   ('display', 0.009024031387935262),
   ('feature', 0.008697073728952101),
  

In [27]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [28]:
# # Can take a long time to run.
# model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=20, step=1)

In [29]:
# # Show graph
# limit=20; start=2; step=1;
# x = range(start, limit, step)
# plt.plot(x, coherence_values)
# plt.xlabel("Num Topics")
# plt.ylabel("Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

In [30]:
# # Download wordnet to find meaning of words, synonyms and antonyms
# nltk.download('wordnet')

# # Function to lemmatize and more words to their root
# def get_lemma(word):
#     lemma = wn.morphy(word)
#     if lemma is None:
#         return word
#     else:
#         return lemma
    
# # Compile set of stopwords
# nltk.download('stopwords')
# en_stop = set(nltk.corpus.stopwords.words('english'))

# def prepare_text_for_lda(text):
#     tokens = tokenize(text)
#     tokens = [token for token in tokens if len(token) > 4]
#     tokens = [token for token in tokens if token not in en_stop]
#     tokens = [get_lemma(token) for token in tokens]
#     return tokens

# text_data = []

# # Prepare training set for LDA
# tokens = json_data['text'].apply(lambda x: prepare_text_for_lda(x))

# # Prepare Dataframe for later
# json_data['text'] = json_data['text'].apply(lambda x: prepare_text_for_lda(x))

# # Append tokenized text to list of tokenized data
# null = tokens.apply(lambda x: text_data.append(x))

# # Assemble tokenized text data into a dictionary
# dictionary = corpora.Dictionary(text_data)

# # Create Bag of Words corpus from text data
# corpus = [dictionary.doc2bow(text) for text in text_data]

# # Create TF-IDF vectors from our bag of words
# tfidf = models.TfidfModel(corpus)
# corpus_tfidf = tfidf[corpus]


# for doc in corpus_tfidf:
#     pprint(doc)
#     break
    
    
# pickle.dump(corpus_tfidf, open('corpus_tfidf.pkl', 'wb'))
# dictionary.save('dictionary.gensim')

In [31]:
    # IPhone X
    # S9
    # Pixel 3
    # Huawei Mate 20 Pro 
    # OnePlus 6T
    # Huawei P2 Pro
    # LG V4 Thinq
    # Sony Xperia XZ3
    # Essential Phone
    # Razor Phone 2
    # HTC U12+
    # Moto G6 plus

In [32]:
####

## Remove custom stop words

## Assign the topics to the texts
## Pickle dump the topic models
## Call the models the in the opinion mining notebook and use them to classify the texts into topics
## Use Multilabel Naive Bayes to get an output of aspect terms in a dataframe grouped into topic columns, ordered by score

### HOW TO EVALUATE YOUR MODEL!!??

## Clean the Notebook, write up a report
## Create Presentation